<a href="https://colab.research.google.com/github/smartgrids-aau/GreenCodesPython/blob/main/Exemple_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

fileName = 'data for 3.5 .xlsx'
sheetName = 'Sheet1'


df = pd.read_excel(fileName, sheet_name=sheetName)


TestdayNumber = df.iloc[5845:6211, 11].values
Gglobal = df.iloc[5845:6211, 4].values
Gdiffused = df.iloc[5845:6211, 5].values

N = TestdayNumber
G = Gglobal
D = Gdiffused


L = 25.0

OptimumB_M = []
data_length = len(G)


days_per_month = data_length // 12

for month in range(1, 13):
    G_M = []
    D_M = []
    N_M = []


    start_idx = (month - 1) * days_per_month
    if month == 12:
        end_idx = data_length
    else:
        end_idx = month * days_per_month

    for j in range(start_idx, end_idx):
        G_M.append(G[j])
        D_M.append(D[j])
        N_M.append(N[j])


    G_M = np.array(G_M)
    D_M = np.array(D_M)
    N_M = np.array(N_M)

    days = len(G_M)
    GBAns = []

    for B in range(0, 91):

        Ds = 23.45 * np.sin((360 * (284 + N_M) / 365) * (np.pi / 180))

        cos_W = -1 * np.tan(L * (np.pi / 180)) * np.tan(Ds * (np.pi / 180))
        cos_W = np.maximum(-1, np.minimum(1, cos_W))
        W = np.arccos(cos_W) * 180 / np.pi

        denominator = (np.cos(L * (np.pi / 180)) * np.cos(Ds * (np.pi / 180)) *
                      np.sin(W * (np.pi / 180))) + \
                     ((W * (np.pi / 180)) * np.sin(L * (np.pi / 180)) *
                      np.sin(Ds * (np.pi / 180)))


        denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)

        Rb = ((np.cos((L - B) * (np.pi / 180)) * np.cos(Ds * (np.pi / 180)) *
               np.sin(W * (np.pi / 180)) +
               (W * (np.pi / 180)) * np.sin((L - B) * (np.pi / 180)) *
               np.sin(Ds * (np.pi / 180)))) / denominator

        Rd = (1 + np.cos(B * (np.pi / 180))) / 2
        Rr = (0.3 * (1 - np.cos(B * (np.pi / 180)))) / 2
        F = G_M - D_M
        BB = F * Rb
        DB = D_M * Rd
        RB = G_M * Rr
        GB = BB + DB + RB

        AV_GB = np.mean(GB)
        GBAns.append(AV_GB)

    GBAns = np.array(GBAns)
    MAX_INDEX = np.argmax(GBAns)
    MAX = GBAns[MAX_INDEX]
    maximum_Solar_Radiation = MAX
    optimumB1 = MAX_INDEX
    OptimumB_M.append(optimumB1)

OptimumB_M = np.array(OptimumB_M)
print("OptimumB_M:", OptimumB_M)

Year_Months = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

# Plotting
plt.figure()
plt.subplot(2, 2, 1)
plt.plot(Year_Months, OptimumB_M, '-kd', linewidth=2.5,
         markeredgecolor='red', markerfacecolor='red', markersize=8)
plt.xlim([1, 12])
plt.legend(['Optimum Tilt angle'], fontsize=8)
plt.xlabel('Month', fontsize=14)
plt.ylabel('Optimum Tilt angle', fontsize=14)
plt.grid(True)
plt.title('Monthly optimum tilt angle', fontsize=14)

plt.gca().tick_params(labelsize=12)
plt.show()


IndexError: single positional indexer is out-of-bounds